# FAA 1462 Pareja 10. Práctica 4 REGRESIÓN LOGÍSTICA Y ANÁLISIS ROC
## Kevin de la Coba Malam

En este documento se hara un análisis del clasificador de **Algoritmos Genéticos** usando diferenrtes *poblaciones*, *generaciones*, *cruces*, *mutaciones* y *elitismo*. También se comparará el algoritmo con el clasificador de  **Naive Bayes** mediante el *espacio ROC*. 

En está practica, se ha usado solo la validación simple. En concreto, con la validación simple se usán valores para el %: **20** (se ejecutará 3 veces). Al tener varios tipos de mutación y cruce el test se hace muy largo.

Para simplificar el documento se ha creado el archivo *utils.py* en el cual se encuentran las funciones para clasificar, mostrar gráficas...

### Modificaciones

- La clase Datos se ha modificado, el constructor tiene un parametro opcional (allNominal) que por defecto está a False para tratar a todos los atributos como si fuesen nominales. 

In [1]:
# Para esconder los warnings
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn

from ClasificadorAlgoritmoGenetico import AlgoritmoGenetico as AG
from Datos import Datos
import utils
import matplotlib.pyplot as plt

# Cargamos datos
l = Datos("lentillas.data")
titanic = Datos("titanic.csv", allNominal=True)
ttt = Datos("tic-tac-toe.data")

## 1. Detalles de la implementación

Lo primero sería mencionar la estructura que tiene cada individuo. Para crear esta estructura se ha hecho una clase interna en la cual se define a un individuo con los siguientes datos.
```py
class Individuo:
	fitnessValue # Valor de fitness del individuo
	priori # Clase mayoritaria de los datos
	reglas = [
		{"regla": [int], "conclusion": int} # Cada regla es un diccionario
		...
	]
```
Para representar las reglas se usa un diccionario, en este diccionario tenemos la clave "regla" en la cual el valor es una lista de enteros. Cada entero de esta lista hace referencia a la cadena de bits para un atributo, por ejemplo: Si tenemos 3 atributos A, B y C, tendremos 3 enteros en la lista, si cada atributo tiene 3 valores, se usaran 3 bits de cada entero. Para operar con estos bits se usan bit operators (&, |).

La conclusión es un entero que marca 0 o 1.

### 1.1 Generación de la población inicial

Las reglas se crean de forma aleatoria, asignando para cada atributo una cadena aleatoria (de la cual solo se van a usar los N primeros bits, siendo N el número de posibles valores en el atributo). Para este análisis se usaran individuos con un **máximo de 5 reglas**.

### 1.2 Cruces implementados

Se han implementado el cruce *inter reglas* y el cruce *intra reglas*. Son métodos estáticos de la clase *ClasificadorAlgoritmoGenetico* que deben pasarse al propio constructor de la clase (patrón template).

### 1.3 Mutaciones implementadas

Se ha implementado la *mutación estandar* y la *mutación de reglas*. De nuevo, son métodos estáticos que deben pasarse al constructor.

### 1.4 Clasificación

La clasificación es simple, se coge al mejor individuo del entrenamiento y se aplican las reglas, pero, ¿cómo se aplican estas reglas? Al utilizar cadenas de bits para cada atributo es simple comparar si un valor se cumple en una regla o no, por ejemplo:
Si tenemos el atributo A con los posibles datos 'a', 'b', 'c', tendríamos una cadena de bits para este atributo con 3 bits. Si recibimos el valor 'a', recibimos la cadena 001 (un entero con valor 1), si recibimos 'b' 010 (entero con valor 2) y si recibimos 'c' 100 (entero con valor 4). Con puertas lógicas es muy simple comparar estos valores. Si tenemos la regla 101 (entero con valor 5) y recibimos el valor 001 (a) (aplicamos la puerta lógica and sobre los valores enteros 5 y 1, 101 & 001), pues esa parte de la regla se aprobaría para este atributo y haría falta hacerlo con todos los demás, si se cumplen todos devolvemos la conclusión.
Si varias reglas se cumplen, escogemos la conclusión mayoritaria, y si no se cumple ninguna regla, devolvemos la clase mayoritaria (a priori).

En el entrenamiento (también en la clasificación) puede pasar que varias relgas se cumplan o que no se cumpla ninguna, en el primer caso lo que se hace es devolver la conclusión mayoritaria, en el segundo caso se devulve la clase a priori (clase mayoritaria).

## 2. Resultados clasificación.

Las opciones que se han usado para la clasificación son las siguientes:
- **Población**: Variable [50-150].
- **Generacion**: Variable [100-200].
- **Máximo número de reglas**: 5.
- **Probabilidad mutación**: 5%.
- **Proporción de elitismo**: 5%.
- **Cruce**: Ambos (inter e intra).
- **Mutación**: Ambas (estandar y de reglas).
- **Método de validación**: Simple 20%.

### Cruce *Inter Reglas* y mutación *estandar*

In [2]:
errorMedioTTT_inter_estandar, errorMedioTITANIC_inter_estandar = utils.AG_test(ttt, titanic, AG.cruceInterReglas, AG.mutacionEstandar)

Test Poblacion=50 Generaciones=100	Tic-Tac-Toe - Error=0.376963		Titanic - Error=0.507177

[Tic-Tac-Toe] reglas del mejor individuo:
 [REGLA 1] - IF TLeftSq=(x) AND TMidSq=(o) AND TRightSq=(b OR o) AND MLeftSq=(b OR x) AND MMidSq=(b OR o) AND MRightSq=(b OR x) AND BLeftSq=(b OR x) AND BMidSq=(b OR x) AND BRightSq=(b) THEN concl=1
[REGLA 2] - IF TLeftSq=(o) AND TMidSq=(b OR o) AND TRightSq=(b OR o) AND MLeftSq=(b OR o) AND MMidSq=(b OR o) AND MRightSq=(x) AND BLeftSq=(b) AND BMidSq=(x) AND BRightSq=(b) THEN concl=0

[Titanic] reglas del mejor individuo:
 [REGLA 1] - IF Pclass=(3) AND Sex=(female) AND Age=(0 OR 1 OR 2 OR 5 OR 7 OR 8 OR 9 OR 10 OR 11) THEN concl=1
[REGLA 2] - IF Pclass=(3) AND Sex=(female) AND Age=(1 OR 2 OR 3 OR 4 OR 5 OR 7 OR 8 OR 9 OR 15) THEN concl=1
[REGLA 3] - IF Pclass=(3) AND Sex=(female) AND Age=(1 OR 2 OR 3 OR 4 OR 5 OR 7 OR 8 OR 9 OR 15) THEN concl=1
[REGLA 4] - IF Pclass=(1 OR 3) AND Sex=(female) AND Age=(0 OR 1 OR 2 OR 3 OR 6 OR 7 OR 8 OR 9 OR 12 OR 13 OR 14 

### Cruce *Inter Reglas* y mutación *de reglas*

In [ ]:
errorMedioTTT_inter_reglas, errorMedioTITANIC_inter_reglas = utils.AG_test(ttt, titanic, AG.cruceInterReglas, AG.mutacionReglas)

Test Poblacion=50 Generaciones=100	Tic-Tac-Toe - Error=0.361257		Titanic - Error=0.368421

[Tic-Tac-Toe] reglas del mejor individuo:
 [REGLA 1] - IF TLeftSq=(b OR o OR x) AND TMidSq=(x) AND ) AND MMidSq=(b OR o) AND MRightSq=(o) AND BLeftSq=(b OR o OR x) AND BMidSq=(x) AND BRightSq=(b OR x) THEN concl=0
[REGLA 2] - IF TLeftSq=(b OR o OR x) AND TMidSq=(b OR x) AND TRightSq=(b OR x) AND MMidSq=(x) AND MRightSq=(o) AND BLeftSq=(b OR o OR x) AND BMidSq=(x) AND BRightSq=(b OR o OR x) THEN concl=0

[Titanic] reglas del mejor individuo:
 [REGLA 1] - IF Sex=(female) AND Age=(0 OR 2 OR 3 OR 4 OR 9 OR 10 OR 12 OR 15) THEN concl=1
[REGLA 2] - IF Pclass=(1 OR 3) AND Age=(0 OR 1 OR 2 OR 9 OR 14 OR 15) THEN concl=1
[REGLA 3] - IF Pclass=(2 OR 3) AND Sex=(female) AND Age=(0 OR 1 OR 2 OR 4 OR 5 OR 6 OR 7 OR 12 OR 15) THEN concl=0

Test Poblacion=50 Generaciones=200	Tic-Tac-Toe - Error=0.366492		Titanic - Error=0.607656

[Tic-Tac-Toe] reglas del mejor individuo:
 [REGLA 1] - IF TLeftSq=(b OR x) AND TMi

### Cruce *Intra Reglas* y mutación *estandar*

In [ ]:
errorMedioTTT_intra_estandar, errorMedioTITANIC_intra_estandar = utils.AG_test(ttt, titanic, AG.cruceIntraReglas, AG.mutacionEstandar)

### Cruce *Intra Reglas* y mutación *de reglas*

In [ ]:
errorMedioTTT_intra_reglas, errorMedioTITANIC_intra_reglas = utils.AG_test(ttt, titanic, AG.cruceIntraReglas, AG.mutacionReglas)

# PONER CONCLUSION APARTADO ANTERIOR. DECIR QUE poblacion y generaciones es mejor, y decir que cruce y que mutacion es mejor.

Podemos ver que tras cada ejecución se muestran las reglas de los mejores individuos en cada una de estas. En estas reglas predomina la **conclusión=1** con muy escasas reglas que contengan una **conclusión=0**. 

Con respecto a los métodos de cruce, no vemos una diferencia notoria en los resultados, lo mismo pasa con la mutación, las diferentes mutaciones no han provocado un cambio notorio en el error.
Lo que si que provoca una diferencia es el número de generaciones, en general cuantas menos generaciones, más error. Pero esta regla no siempre se cumple debido a la aleatoriedad del algoritmo, puede pasar que en la generación 2, se cree un individuo el cuál tiene un fitness muy alto.

Podemos concluir que tenemos un error medio en el dataset de **Tic-Tac-Toe del 30%** y en del **Titanic del 40%-45%**.


## 3. Análisis
Lo primero que destaca a la vista es el pronunciado error. Una de las posibles razones es por **la validación en si**, al no entrenar con todos los datos puede que las reglas se *sobreajusten* a los datos (el fitness del mejor individuo suele rondar el 70%-90% de aciertos, puede que se sobreajuste a los datos de entrenamiento), puede que el mejor individuo tras el entrenamiento tenga reglas que no se adecúen correctamente a los datos de test, esto provoca que se recurra mucho al priori provocando quizás mas errores de los deseados. Otra razón puede ser el **número de reglas**, este puede ser un factor clave en relación al error obtenido. Por último, otra razón que se me ocurre es por el hecho de que el algoritmo en si **es muy aleatorio**, puede que esa aleatoriedad no permita una clasificación mejor.

Algo que se puede ver, es que en algunos individuos se repiten reglas, asumo que esto significa que estas reglas son buenas, ya que al cruzar individuos se han mantenido durante generaciones. Pero esto también tiene otra explicación, y es el propio método de cruce, el *cruce inter reglas* cruza las reglas de dos individuos **sin modificarlas**, esto puede causar lo anteriormente mencionado.

Otro factor relevante es la *probabilidad de mutación*. Una probabilidad mayor podría acelerar el proceso de encontrar un muy buen individuo, pero hay que tener cuidado, una probabilidad muy grande puede provocar más errores, es muy similar al descenso de gradiente en regresión logística, si la constante de aprendizaje era muy alta, puede que no encontrasemos nunca mínimos, lo mismo pasa aquí, una probabilidad muy alta de mutación puede que lastre al algoritmo.

# CHECKEAR CUANDO ACABE
**En el cruce intra reglas** no vemos tanta repetición de reglas.

## 4. Evolución del fitness
### 4.1 Titanic

In [ ]:
# Plot de 2 fitness maximo y medio, en cada plot varios runs
utils.plot(titanic, "Titanic", AG.cruceInterReglas, AG.mutacionReglas)

hablar de el crecimiento.

### 4.2 Tic-Tac-Toe

In [ ]:
# Plot de 2 fitness maximo y medio, en cada plot varios runs
utils.plot(ttt, "Tic-Tac-Toe", AG.cruceInterReglas, AG.mutacionReglas)

Hablar del crecimiento.

## 5. Espacio ROC 
### 5.1 Titanic

In [ ]:
# Generar espacio ROC con naive bayes
utils.plot_espacio_ROC(titanic, 5, 20, AG.cruceInterReglas, AG.mutacionReglas)

Conclusion